# Geocoding addresses using the geoadmin API and Python

## Libraries and settings

In [ ]:
# Libraries
import os
import requests
import json
import urllib
import fnmatch
import folium
import platform
import pandas as pd
import geopandas as gpd
from IPython.display import clear_output

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

## Geocoding a single address

### Define base url for address search

In [ ]:
# Define base url for address search
base_url= "https://api3.geo.admin.ch/rest/services/api/SearchServer?"

# Set up search parameters: address, origins and type
parameters = {"searchText": "8400 Winterthur, Theaterstrasse 17",
              "origins": "address",
              "type": "locations",
             }

# Urllib.parse.urlencode turns parameters into url
# print(f"{base_url}{urllib.parse.urlencode(parameters)}")

### Server request & response

In [ ]:
# Server request
r = requests.get(f"{base_url}{urllib.parse.urlencode(parameters)}")

# Get data in json-format
data = json.loads(r.content)
data

# Take only the first server response, convert to data frame with relevant infos
df = pd.DataFrame.from_dict(list(data.values())[0][0], orient='columns')
df.iloc[[1,4,5,6,11,12],:1]

## Geocoding multiple addresses

### Importing apartment data

In [ ]:
# Get current working directory
print(os.getcwd())

# Show all files in the directory
flist = fnmatch.filter(os.listdir('.'), '*.csv')
for i in flist:
    print(i)

# Read the data to a pandas data frame
df = pd.read_csv('apartments_data_prepared.csv', 
                 sep=',', 
                 encoding='utf-8')[['web-scraper-order', 
                                    'address_raw', 
                                    'datetime', 
                                    'rooms', 
                                    'area', 
                                    'luxurious', 
                                    'price_per_m2']][:100] # first 100 records

# Get number of rows and columns
print(df.shape)

# Show first records
df.head(5)

### Geocoding addresses using the geoadmin API

In [ ]:
# Define base url
base_url= "https://api3.geo.admin.ch/rest/services/api/SearchServer?"

# Geocode list of adresses
geolocation = []
n = 1
for i in df['address_raw'].astype(str):
    
    print('Geocoding address', 
          n, 
          'out of', 
          len(df['address_raw']), 
          ':', 
          i)
    n=n+1
    clear_output(wait=True)
    
    try:
        # Set up search parameters - address, origins and type
        parameters = {"searchText": i,
                      "origins": "address",
                      "type": "locations",
                     }

        # Server request
        r = requests.get(f"{base_url}{urllib.parse.urlencode(parameters)}")

        # Get data
        data = json.loads(r.content)

        # Take first server response, convert to df with relevant infos
        df_loc = pd.DataFrame.from_dict(list(data.values())[0][0], 
                                        orient='columns')
        geolocation.append(df_loc.iloc[[5,6],0].astype(float))
    
    except:
        geolocation.append(pd.Series(data={'lat': None, 'lon': None}))
        
# Write lat and lon to df
df_loc = pd.DataFrame(geolocation, 
                      columns=("lat", "lon"), 
                      index=range(len(df['address_raw'])))
df['lat'] = df_loc['lat']
df['lon'] = df_loc['lon']
df.head(5)

### Read polygon-map with municipalities of the canton of Zuerich

In [ ]:
# Polygonmap als .json-File (WGS84)
polys = gpd.read_file("GEN_A4_GEMEINDEN_2019_epsg4326.json")
print(type(polys))
polys.head(5)

### Plot map

In [ ]:
# Initialisierung der Map
m = folium.Map(location=[47.44, 8.65], zoom_start=10)

# Map settings
folium.Choropleth(
    geo_data=polys,
    name='polys',
    fill_color='greenyellow'
).add_to(m)

# Add lat/lon of addresses
df_sub = df.dropna()
for i in range(0, len(df_sub)):
    folium.Marker(location=(df_sub.iloc[i]['lat'], 
                            df_sub.iloc[i]['lon']), 
                  popup=df_sub.iloc[i]['address_raw']).add_to(m)

# Layer control
folium.LayerControl().add_to(m)

# Plot map
m

### Intersect municipality polygon-map with lat and lon (point-in-polygon intersection)

In [ ]:
# lat/lon to GeoDataFrame
pnts = gpd.GeoDataFrame(df, 
                        geometry = gpd.points_from_xy(df['lon'], 
                                                      df['lat']))

pnts

# Merge spatial data
data_merged = gpd.sjoin(pnts, polys, how="inner", op='within')

# Get relevant columns
df2 = data_merged[['web-scraper-order', 
                   'address_raw', 
                   'lat',
                   'lon',
                   'BFS', 
                   'NAME']]
df2 = df2.rename(columns = {'BFS': 'bfs_number', 
                            'NAME': 'bfs_name'})
df2.head(5)

### Save data to file

In [ ]:
df2.to_csv('apartments_data_geocoded.csv', 
           sep=",", 
           encoding='utf-8',
           index=False)

### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')